#### Interaction

In [1]:
from plip.structure.preparation import PDBComplex
from openbabel import pybel
from openbabel import openbabel as ob
import numpy as np
import subprocess
import os
import shutil
import glob
import copy
import multiprocessing as mp
import os
import pickle
import random
import sys
import time
from collections import Counter
import os, sys
"""
your path 
sys.path.append(" ")
"""
sys.path.append('.')
import numpy as np
import pandas as pd
import rdkit
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
import torch
import tqdm
from biopandas.pdb import PandasPdb
from mol_tree import *
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol
from rdkit.Chem import rdDepictor
from rdkit.Chem import rdDistGeom
import rdkit
from rdkit.Chem import Draw
from PIL import Image
from io import BytesIO
print(rdkit.__version__)

2023.03.2


In [2]:
"""
data_prfix : path of "crossdocked_v1.1_rmsd1.0_pocket10"
"""
data_prefix = '/home/user/work/3D/PharDiff/data/crossdocked_v1.1_rmsd1.0_pocket10'
complex_path = data_prefix.replace("crossdocked_v1.1_rmsd1.0_pocket10", "complex")

complex_dir = glob.glob(complex_path+"/*")

complex_pdb_list = []
for com_dir in complex_dir:
    for mono_pair in glob.glob(com_dir+"/*complex"):
        #print(glob.glob(mono_pair+"/*"))
        complex_pdb_list.append(glob.glob(mono_pair+"/*")[0])


In [ ]:
def extract_name(classname):
    return classname.split('.')[-1].split("'")[0]
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)
#data_dir = '/home/csy/work/3D/targetdiff_clone/data/'
#pocket_pdb_list = os.listdir(data_dir + 'convert')
whole_ = []
vocab_super = dict()
err = 0
cnt = 0
from tqdm import tqdm
for proc, pair in enumerate(complex_pdb_list):
    """
    binding site indentification using PLIP
    """    
    mol = PDBComplex()
    mol.load_pdb(pair)
    mol.analyze()                        

    longnames = [x.longname for x in mol.ligands]
    bsids = [":".join([x.hetid, x.chain, str(x.position)]) for x in mol.ligands]
    indices = [j for j,x in enumerate(longnames) if x == 'ISK']  

    # Interaction Parsing !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    inter_list = []
    for bs in bsids:
        interactions = mol.interaction_sets[bs]
        for inter in interactions.all_itypes:
            
            # Hydroph_interaction
            if extract_name(str(inter.__class__)) == 'hydroph_interaction':
                inter_dict = inter._asdict()
                inter_dict["interaction"] = extract_name(str(inter.__class__))
                inter_dict["sdf_idx"] = inter.ligatom_orig_idx -1 - min(mol.ligands[0].can_to_pdb.values())
                inter_list.append(inter_dict)

            # Hydrogen bond
            if extract_name(str(inter.__class__)) == 'hbond':
                inter_dict = inter._asdict()
                inter_dict["interaction"] = extract_name(str(inter.__class__))
                
                if inter.d_orig_idx > min(mol.ligands[0].can_to_pdb.values()):
                    inter_dict["sdf_idx"] = inter.d_orig_idx -1 - min(mol.ligands[0].can_to_pdb.values())
                else:
                    inter_dict["sdf_idx"] = inter.a_orig_idx -1 - min(mol.ligands[0].can_to_pdb.values())    
                    
                inter_list.append(inter_dict)
            
            # Pi-Stacking 
            if extract_name(str(inter.__class__)) == 'pistack':
                inter_dict = inter._asdict()
                inter_dict["interaction"] = extract_name(str(inter.__class__))
                inter_dict["sdf_idx"] = [idx - 1 - min(mol.ligands[0].can_to_pdb.values()) for idx in inter_dict['ligandring'].atoms_orig_idx]
                inter_list.append(inter_dict) 
         
            # Halogen bonding
            if extract_name(str(inter.__class__)) == 'halogenbond':
                inter_dict = inter._asdict()
                inter_dict["interaction"] = extract_name(str(inter.__class__))
                inter_dict["sdf_idx"] = inter.don.x_orig_idx -1 - min(mol.ligands[0].can_to_pdb.values())
                inter_list.append(inter_dict)                
                
            # Pi-Cation interaction    
            if extract_name(str(inter.__class__)) == 'pication':
                inter_dict = inter._asdict()
                inter_dict["interaction"] = extract_name(str(inter.__class__))        
                
                # charge쪽이 ligand
                if min(inter.charge.atoms_orig_idx) > min(mol.ligands[0].can_to_pdb.values()):
                    inter_dict["sdf_idx"] = [idx -1 - min(mol.ligands[0].can_to_pdb.values()) for idx in inter_dict['charge'].atoms_orig_idx]
                
                # ring쪽이 ligand 
                else:
                    inter_dict["sdf_idx"] = [idx -1 - min(mol.ligands[0].can_to_pdb.values()) for idx in inter_dict['ring'].atoms_orig_idx]

                inter_list.append(inter_dict) 
 
                
    """
    Ligand sdf parsing
    """             

    vocab_dir = '/home/user/work/3D/PharDiff/datasets/vocab_.txt'

    with open(vocab_dir, 'r') as f:
        vocab = [x.strip() for x in f.readlines()]    
      
    vocab_fp = pd.read_csv('/home/user/work/3D/PharDiff/datasets/vocab_blur_fps_updated.csv', index_col=0)
    vocab = Vocab(vocab)           
    temp = pair.split('/')
    temp_dir = os.path.join('/'.join(temp[:-2]), temp[-1].replace("complex_", "").replace(".pdb", ".sdf"))
    crossdock_dir_sdf = temp_dir.replace('complex', 'crossdocked_v1.1_rmsd1.0_pocket10')
    suppl = Chem.SDMolSupplier(crossdock_dir_sdf)
    mol_list = [x for x in suppl if x is not None]    
    for i, mol in enumerate(mol_list):
        tree_list = []
        try:
            jt = MolTree(mol, vocab=vocab)
            tree_list.append(jt)  
                
                
            pairbynode_smi = []
            for inter in inter_list:
                interbynode=[]
                    
                for node in jt.nodes:
                    
                    if (inter['interaction'] == 'hydroph_interaction' or 'halogenbond' or 'hbond'):
                        if (inter["sdf_idx"] in node.clique) == True:
                            interbynode.append(node.clique)
                            
                    if inter['interaction'] == 'pistack':
                        if (sorted(inter["sdf_idx"]) == sorted(node.clique)) == True:
                            interbynode.append(node.clique)
                            
                    if inter['interaction'] == 'pication':

                        if len(inter["sdf_idx"]) > 1:
                            if (sorted(inter["sdf_idx"]) == sorted(node.clique)) == True:
                                interbynode.append(node.clique)  
                
                        else:
                            if (inter["sdf_idx"][0] in node.clique) == True:
                                interbynode.append(node.clique)  
                                        
                if not len(interbynode) == 0: 
                    try: 
                        a = Chem.MolFromSmiles(Chem.MolFragmentToSmiles(jt.mol3D, sum(interbynode, []), kekuleSmiles=True))
                    except Exception as e:
                        err += 1
                        print(e)
                        print("KekulizeException: " , err)
                    smi = Chem.MolFragmentToSmiles(jt.mol3D, sum(interbynode, []), kekuleSmiles=True)
                    pairbynode_smi.append(a)
                    if (smi in vocab_super.keys()) == True: 
                        vocab_super[smi] += 1
                    else:
                        vocab_super[smi] = 1
        except Exception as e:
            print("MolTree Error Count : ", cnt)
            print(e)
            cnt += 1 
            continue
        
    if proc % 5000 == 0 :
        print(err)
        print(len(vocab_super))

In [ ]:
# with open('frag_vocab.pickle','wb') as fw:
#     pickle.dump(vocab_super, fw)
import pickle
with open('/home/csy/work/3D/targetdiff_phar/datasets/frag_vocab.pickle', 'rb') as fr:
    user_loaded = pickle.load(fr)